In [1]:
import cv2
import numpy as np
import os
from PIL import Image
from typing import List
import json
import sys

In [2]:
def get_path_in_parent(*args):
    return os.path.abspath(os.path.join(os.getcwd(), '..', *args))

In [3]:
def get_outputs(model_num: int) -> List[np.ndarray]:
    net = cv2.dnn.readNet(
        get_path_in_parent(f"yolo_model_{model_num}", f"yolov4-tiny-logistics_size_416_{model_num}.weights"),
        get_path_in_parent(f"yolo_model_{model_num}", f"yolov4-tiny-logistics_size_416_{model_num}.cfg")
    )
    
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
      
    outputs = []
    logistics_path = get_path_in_parent("logistics")
    for filename in sorted(os.listdir(logistics_path)):
        if not filename.lower().endswith(".jpg"):
            continue
        
        image = np.array(Image.open(os.path.join(logistics_path, filename)))
        
        blob = cv2.dnn.blobFromImage(image,
                                     scalefactor = 1 / 255.,
                                     size=(416, 416),
                                     mean=(0, 0, 0),
                                     swapRB=True,
                                     crop=False)
        
        net.setInput(blob)
        
        outputs.append(net.forward(output_layers))
        
    return outputs

In [4]:
outputs_1 = get_outputs(1)

In [5]:
# Convert to json serializable
for i, feature_map in enumerate(outputs_1):
    outputs_1[i] = [detection.tolist() for detection in feature_map]

In [9]:
with open(get_path_in_parent("results", "model_1.json"), "w") as f:
    json.dump({
        1: outputs_1
    }, f, indent=2)

In [ ]:
outputs_2 = get_outputs(2)

: 

In [ ]:
# Convert to json serializable
for i, feature_map in enumerate(outputs_2):
    outputs_2[i] = [detection.tolist() for detection in feature_map]

In [ ]:
with open(get_path_in_parent("results", "model_2.json"), "w") as f:
    json.dump(outputs_2, f, indent=2)